## Data Analysis Pipeline Steps
1. Import & concatenate data:
- Write a function that takes a folder of data files as an argument and concatenates all the .csv files in that folder into a single pandas dataframe.
    - *hint*: use the packages "glob" and "pandas"
    - *hint2*: you need to skip the first three rows of each .csv file for this to work.
2. Average response score for each object.
- Get a summary of the average score for each object shown.
    - *hint*: There are a couple of ways to do this, you can use the *.groupby()* method in pandas to group items in the 'object_shown' column, or you can create a pivot table.
- Check whether any objects have average response scores lower than 6.
3. Find the images we want to use.
- Group by images (in the 'stim1' column) and get the average score for each stim1. 
    - *hint* use the .groupby() and .mean() methods
- For each object, find the two highest-scored images that have the same score. 
    - If no two objects have the same score, list the two highest scores for each object. 
    - If more than two objects have matching highest-scores, list all of them.

In [80]:
import pandas as pd
import numpy as np
from glob import glob

In [81]:
def import_all_csvs(folder):
	files = glob(folder + '/*.csv')
	master_df = pd.concat([pd.read_csv(f, skiprows=[0,1,2,4,5]) for f in files ])
	return master_df

In [82]:
data = import_all_csvs('/Users/sheaduarte/Desktop/Projects/MSImemory/MS-Memory-Analysis/Memory_Data/data/raw/typicality_test_data')

In [83]:
data.head()

,rowNo,type,stim1,stimFormat,ITI,responseType,responseOptions,trialText,head,required,object_shown,seperator,ITI_ms,ITI_f,ITI_fDuration,timestamp,response,RT,responseCode
0,7,test,goose_1,.png,800,slider,0_10_1,How typical is this orientation?,NaN,NaN,goose,NaN,800,48,16.67,513338,9,27561,9
1,8,test,goose_2,.png,800,slider,0_10_1,How typical is this orientation?,NaN,NaN,goose,NaN,800,48,16.67,557605,8,43472,8
2,9,test,goose_3,.png,800,slider,0_10_1,How typical is this orientation?,NaN,NaN,goose,NaN,800,48,16.67,650207,6,91807,6
3,10,test,goose_4,.png,800,slider,0_10_1,How typical is this orientation?,NaN,NaN,goose,NaN,801,48,16.70,659106,8,8102,8
4,11,test,goose_5,.png,800,slider,0_10_1,How typical is this orientation?,NaN,NaN,goose,NaN,800,48,16.67,698737,6,38834,6


In [84]:
object_means_table = data.pivot_table(values = 'response',index='object_shown')

Groupby stim1, find mean of each, round.
for each object_shown, find the two highest max image scores, 
if they're the same, choose those images

In [85]:
score_pivot = data.pivot_table(index='stim1', values = 'response').round(0)
display(score_pivot)

,response
stim1,
bat_1,6.0
bat_2,4.0
bat_3,5.0
bat_4,5.0
bat_5,5.0
...,...
wrench_1,6.0
wrench_2,6.0
wrench_3,5.0


In [86]:
score_pivot = score_pivot.reset_index()

In [87]:
object_list = list(data['object_shown'].unique())

In [88]:
print(object_list)

['goose', 'record_player', 'crocodile', 'door', 'butterfly', 'smartphone', 'cow', 'horse', 'camera', 'rat', 'cat', 'car', 'rabbit', 'saw', 'frog', 'boat', 'whale', 'bear', 'fish', 'elephant', 'bell', 'wolf', 'tape', 'piano', 'guitar', 'radio', 'train', 'leopard', 'bird', 'cymbal_ride', 'chicken', 'bus', 'walkie', 'pig', 'stapler', 'deer', 'watering_can', 'salamander', 'drum', 'snake', 'crab', 'penguin', 'keyboard', 'drill', 'bicycle', 'laptop', 'camel', 'coins', 'hammer', 'jet', 'dog', 'scissors', 'goat', 'motorcycle', 'rhino', 'shark', 'bat', 'washer', 'tiger', 'wrench']


In [89]:
score_pivot['object_image'] = [x[:-2] for x in score_pivot['stim1']]

In [90]:
#display(score_pivot.tail(50))

In [91]:
sorted_df = score_pivot.sort_values(['object_image','response'],ascending=False).groupby('object_image').head()

In [92]:
sorted_df = sorted_df.reset_index()

In [93]:
sorted_df1 = sorted_df[sorted_df.index % 5 == 0]
sorted_df1 = sorted_df1.rename(columns={"response": "response1"})

In [94]:
sorted_df2 = sorted_df[sorted_df.index % 5 == 1]
sorted_df2 = sorted_df2.rename(columns={'stim1':'stim2',"response": "response2"})

In [95]:
stim_df = pd.merge(sorted_df1,sorted_df2, on='object_image', how='right')
stim_df = stim_df[['object_image','stim1','response1', 'stim2','response2']]
display(stim_df)

,object_image,stim1,response1,stim2,response2
0,wrench,wrench_1,6.0,wrench_2,6.0
1,wolf,wolf_1,7.0,wolf_4,7.0
2,whale,whale_5,7.0,whale_2,6.0
3,watering_can,watering_can_1,6.0,watering_can_5,6.0
4,washer,washer_2,8.0,washer_4,7.0
5,walkie,walkie_1,7.0,walkie_4,6.0
6,train,train_1,7.0,train_2,7.0
7,tiger,tiger_1,6.0,tiger_2,6.0
8,tape,tape_1,8.0,tape_5,7.0
9,stapler,stapler_1,8.0,stapler_3,7.0


In [96]:
control_items = ['wrench','whale','walkie','watering_can','tape','shark','salamander','rhino','record_player','radio','rabbit','laptop','fish','crab','coins','camel','butterfly','bus']

In [97]:
all_objects = list(stim_df['object_image'])
animate = ['wolf', 'whale', 'tiger', 'snake', 'shark', 'salamander', 'rhino', 'rat', 'rabbit', 'pig', 'penguin', 'leopard', 'horse', 'goose', 'goat', 'frog', 'fish', 'elephant', 'dog', 'deer', 'crocodile', 'crab', 'cow', 'chicken', 'cat', 'camel','butterfly','bird','bear','bat']
inanimate_df = stim_df[stim_df['object_image'].isin(animate) == False]
inanimate = list(inanimate_df['object_image'])
inanimate

['wrench',
 'watering_can',
 'washer',
 'walkie',
 'train',
 'tape',
 'stapler',
 'smartphone',
 'scissors',
 'saw',
 'record_player',
 'radio',
 'piano',
 'motorcycle',
 'laptop',
 'keyboard',
 'jet',
 'hammer',
 'guitar',
 'drum',
 'drill',
 'door',
 'cymbal_ride',
 'coins',
 'car',
 'camera',
 'bus',
 'boat',
 'bicycle',
 'bell']

In [98]:
control_images = stim_df[stim_df['object_image'].isin(control_items)]
len(control_images)
control_images

,object_image,stim1,response1,stim2,response2
0,wrench,wrench_1,6.0,wrench_2,6.0
2,whale,whale_5,7.0,whale_2,6.0
3,watering_can,watering_can_1,6.0,watering_can_5,6.0
5,walkie,walkie_1,7.0,walkie_4,6.0
8,tape,tape_1,8.0,tape_5,7.0
12,shark,shark_2,7.0,shark_3,6.0
15,salamander,salamander_3,7.0,salamander_4,6.0
16,rhino,rhino_2,7.0,rhino_4,7.0
17,record_player,record_player_1,8.0,record_player_3,8.0
19,radio,radio_2,8.0,radio_4,7.0


In [99]:
use_images = stim_df[stim_df['object_image'].isin(control_items) == False]
use_images=use_images.reset_index(drop=True)
use_images

,object_image,stim1,response1,stim2,response2
0,wolf,wolf_1,7.0,wolf_4,7.0
1,washer,washer_2,8.0,washer_4,7.0
2,train,train_1,7.0,train_2,7.0
3,tiger,tiger_1,6.0,tiger_2,6.0
4,stapler,stapler_1,8.0,stapler_3,7.0
5,snake,snake_1,7.0,snake_2,7.0
6,smartphone,smartphone_2,8.0,smartphone_1,7.0
7,scissors,scissors_1,7.0,scissors_2,7.0
8,saw,saw_1,7.0,saw_3,7.0
9,rat,rat_1,6.0,rat_2,6.0


Make an encoding df, then make a retrieval df, concat the dfs, make one all the way through then make the others.
enc: I can subset this into 3 dfs, add the sounds into each, then concat them back together. Each one of these dfs will be each condition. I want 7 of each level of animacy, so I can first subset into 2 dfs based on animacy, then add 7 cols from each animacy df to df1, then df2, then df3. 

# Counterbalancing conditions

In [111]:
np.random.seed(123)
enc = pd.DataFrame()
enc['image_shown'] = use_images['object_image']
enc['stim1'] = use_images['stim1']
enc['animacy'] = enc['image_shown'].isin(animate)
enc['animacy'] = enc['animacy'].replace({True: 'animate', False: 'inanimate'})
#enc = enc.sample(frac = 1)
enc = enc.reset_index(drop=True)
enc

,image_shown,stim1,animacy
0,wolf,wolf_1,animate
1,washer,washer_2,inanimate
2,train,train_1,inanimate
3,tiger,tiger_1,animate
4,stapler,stapler_1,inanimate
5,snake,snake_1,animate
6,smartphone,smartphone_2,inanimate
7,scissors,scissors_1,inanimate
8,saw,saw_1,inanimate
9,rat,rat_1,animate


In [117]:
anim_df = enc[enc['animacy']=='animate']
anim_df = anim_df.reset_index(drop=True)
inanim_df = enc[enc['animacy']=='inanimate']
inanim_df = inanim_df.reset_index(drop=True)
anim_df

,image_shown,stim1,animacy
0,wolf,wolf_1,animate
1,tiger,tiger_1,animate
2,snake,snake_1,animate
3,rat,rat_1,animate
4,pig,pig_3,animate
5,penguin,penguin_1,animate
6,leopard,leopard_1,animate
7,horse,horse_1,animate
8,goose,goose_1,animate
9,goat,goat_3,animate


In [118]:
inanim_df

,image_shown,stim1,animacy
0,washer,washer_2,inanimate
1,train,train_1,inanimate
2,stapler,stapler_1,inanimate
3,smartphone,smartphone_2,inanimate
4,scissors,scissors_1,inanimate
5,saw,saw_1,inanimate
6,piano,piano_1,inanimate
7,motorcycle,motorcycle_1,inanimate
8,keyboard,keyboard_2,inanimate
9,jet,jet_1,inanimate


In [119]:
group1 = anim_df[0:7].append(inanim_df[0:7])
group2 = anim_df[7:14].append(inanim_df[7:14])
group3 = anim_df[14:].append(inanim_df[14:])

In [125]:
len(group3)

14

In [126]:
group2

,image_shown,stim1,animacy
7,horse,horse_1,animate
8,goose,goose_1,animate
9,goat,goat_3,animate
10,frog,frog_1,animate
11,elephant,elephant_1,animate
12,dog,dog_2,animate
13,deer,deer_4,animate
7,motorcycle,motorcycle_1,inanimate
8,keyboard,keyboard_2,inanimate
9,jet,jet_1,inanimate


In [127]:
group3

,image_shown,stim1,animacy
14,crocodile,crocodile_2,animate
15,cow,cow_2,animate
16,chicken,chicken_1,animate
17,cat,cat_1,animate
18,bird,bird_1,animate
19,bear,bear_1,animate
20,bat,bat_1,animate
14,door,door_1,inanimate
15,cymbal_ride,cymbal_ride_2,inanimate
16,car,car_3,inanimate


## Congruent

In [128]:
enc_cong1 = pd.DataFrame()
enc_cong1 = enc_cong1.append(group1)
enc_cong1['condition'] = 'congruent'
enc_cong1['audio'] = enc_cong1['image_shown'] + 'A.mp3'
enc_cong1

,image_shown,stim1,animacy,condition,audio
0,wolf,wolf_1,animate,congruent,wolfA.mp3
1,tiger,tiger_1,animate,congruent,tigerA.mp3
2,snake,snake_1,animate,congruent,snakeA.mp3
3,rat,rat_1,animate,congruent,ratA.mp3
4,pig,pig_3,animate,congruent,pigA.mp3
5,penguin,penguin_1,animate,congruent,penguinA.mp3
6,leopard,leopard_1,animate,congruent,leopardA.mp3
0,washer,washer_2,inanimate,congruent,washerA.mp3
1,train,train_1,inanimate,congruent,trainA.mp3
2,stapler,stapler_1,inanimate,congruent,staplerA.mp3


In [129]:
enc_cong2 = pd.DataFrame()
enc_cong2 = enc_cong2.append(group2)
enc_cong2['condition'] = 'congruent'
enc_cong2['audio'] = enc_cong2['image_shown'] + 'A.mp3'
enc_cong2

,image_shown,stim1,animacy,condition,audio
7,horse,horse_1,animate,congruent,horseA.mp3
8,goose,goose_1,animate,congruent,gooseA.mp3
9,goat,goat_3,animate,congruent,goatA.mp3
10,frog,frog_1,animate,congruent,frogA.mp3
11,elephant,elephant_1,animate,congruent,elephantA.mp3
12,dog,dog_2,animate,congruent,dogA.mp3
13,deer,deer_4,animate,congruent,deerA.mp3
7,motorcycle,motorcycle_1,inanimate,congruent,motorcycleA.mp3
8,keyboard,keyboard_2,inanimate,congruent,keyboardA.mp3
9,jet,jet_1,inanimate,congruent,jetA.mp3


In [130]:
enc_cong3 = pd.DataFrame()
enc_cong3 = enc_cong3.append(group3)
enc_cong3['condition'] = 'congruent'
enc_cong3['audio'] = enc_cong3['image_shown'] + 'A.mp3'
enc_cong3

,image_shown,stim1,animacy,condition,audio
14,crocodile,crocodile_2,animate,congruent,crocodileA.mp3
15,cow,cow_2,animate,congruent,cowA.mp3
16,chicken,chicken_1,animate,congruent,chickenA.mp3
17,cat,cat_1,animate,congruent,catA.mp3
18,bird,bird_1,animate,congruent,birdA.mp3
19,bear,bear_1,animate,congruent,bearA.mp3
20,bat,bat_1,animate,congruent,batA.mp3
14,door,door_1,inanimate,congruent,doorA.mp3
15,cymbal_ride,cymbal_ride_2,inanimate,congruent,cymbal_rideA.mp3
16,car,car_3,inanimate,congruent,carA.mp3


## Incongruent

In [131]:
enc_incong1 = pd.DataFrame()
enc_incong1 = enc_incong1.append(group1)
enc_incong1['condition'] = 'incongruent'
incong1_audio = list(enc_cong3['audio'])
enc_incong1['audio'] = incong1_audio
enc_incong1

,image_shown,stim1,animacy,condition,audio
0,wolf,wolf_1,animate,incongruent,crocodileA.mp3
1,tiger,tiger_1,animate,incongruent,cowA.mp3
2,snake,snake_1,animate,incongruent,chickenA.mp3
3,rat,rat_1,animate,incongruent,catA.mp3
4,pig,pig_3,animate,incongruent,birdA.mp3
5,penguin,penguin_1,animate,incongruent,bearA.mp3
6,leopard,leopard_1,animate,incongruent,batA.mp3
0,washer,washer_2,inanimate,incongruent,doorA.mp3
1,train,train_1,inanimate,incongruent,cymbal_rideA.mp3
2,stapler,stapler_1,inanimate,incongruent,carA.mp3


In [132]:
enc_incong2 = pd.DataFrame()
enc_incong2 = enc_incong2.append(group2)
enc_incong2['condition'] = 'incongruent'
incong2_audio = list(enc_cong1['audio'])
enc_incong2['audio'] = incong2_audio
enc_incong2

,image_shown,stim1,animacy,condition,audio
7,horse,horse_1,animate,incongruent,wolfA.mp3
8,goose,goose_1,animate,incongruent,tigerA.mp3
9,goat,goat_3,animate,incongruent,snakeA.mp3
10,frog,frog_1,animate,incongruent,ratA.mp3
11,elephant,elephant_1,animate,incongruent,pigA.mp3
12,dog,dog_2,animate,incongruent,penguinA.mp3
13,deer,deer_4,animate,incongruent,leopardA.mp3
7,motorcycle,motorcycle_1,inanimate,incongruent,washerA.mp3
8,keyboard,keyboard_2,inanimate,incongruent,trainA.mp3
9,jet,jet_1,inanimate,incongruent,staplerA.mp3


In [133]:
enc_incong3 = pd.DataFrame()
enc_incong3 = enc_incong3.append(group3)
enc_incong3['condition'] = 'incongruent'
incong3_audio = list(enc_cong2['audio'])
enc_incong3['audio'] = incong3_audio
enc_incong3

,image_shown,stim1,animacy,condition,audio
14,crocodile,crocodile_2,animate,incongruent,horseA.mp3
15,cow,cow_2,animate,incongruent,gooseA.mp3
16,chicken,chicken_1,animate,incongruent,goatA.mp3
17,cat,cat_1,animate,incongruent,frogA.mp3
18,bird,bird_1,animate,incongruent,elephantA.mp3
19,bear,bear_1,animate,incongruent,dogA.mp3
20,bat,bat_1,animate,incongruent,deerA.mp3
14,door,door_1,inanimate,incongruent,motorcycleA.mp3
15,cymbal_ride,cymbal_ride_2,inanimate,incongruent,keyboardA.mp3
16,car,car_3,inanimate,incongruent,jetA.mp3


## Control

In [134]:
jumble_list = ['jumble1.mp3', 'jumble2.mp3','jumble3.mp3','jumble4.mp3','jumble5.mp3','jumble6.mp3','jumble7.mp3','jumble8.mp3','jumble9.mp3','jumble10.mp3','jumble11.mp3','jumble12.mp3','jumble13.mp3','jumble14.mp3']

In [135]:
enc_ctrl1 = pd.DataFrame()
enc_ctrl1 = enc_ctrl1.append(group1)
enc_ctrl1['condition'] = 'control'
enc_ctrl1['audio'] = jumble_list
enc_ctrl1

,image_shown,stim1,animacy,condition,audio
0,wolf,wolf_1,animate,control,jumble1.mp3
1,tiger,tiger_1,animate,control,jumble2.mp3
2,snake,snake_1,animate,control,jumble3.mp3
3,rat,rat_1,animate,control,jumble4.mp3
4,pig,pig_3,animate,control,jumble5.mp3
5,penguin,penguin_1,animate,control,jumble6.mp3
6,leopard,leopard_1,animate,control,jumble7.mp3
0,washer,washer_2,inanimate,control,jumble8.mp3
1,train,train_1,inanimate,control,jumble9.mp3
2,stapler,stapler_1,inanimate,control,jumble10.mp3


In [136]:
enc_ctrl2 = pd.DataFrame()
enc_ctrl2 = enc_ctrl2.append(group2)
enc_ctrl2['condition'] = 'control'
enc_ctrl2['audio'] = jumble_list
enc_ctrl2

,image_shown,stim1,animacy,condition,audio
7,horse,horse_1,animate,control,jumble1.mp3
8,goose,goose_1,animate,control,jumble2.mp3
9,goat,goat_3,animate,control,jumble3.mp3
10,frog,frog_1,animate,control,jumble4.mp3
11,elephant,elephant_1,animate,control,jumble5.mp3
12,dog,dog_2,animate,control,jumble6.mp3
13,deer,deer_4,animate,control,jumble7.mp3
7,motorcycle,motorcycle_1,inanimate,control,jumble8.mp3
8,keyboard,keyboard_2,inanimate,control,jumble9.mp3
9,jet,jet_1,inanimate,control,jumble10.mp3


In [137]:
enc_ctrl3 = pd.DataFrame()
enc_ctrl3 = enc_ctrl3.append(group3)
enc_ctrl3['condition'] = 'control'
enc_ctrl3['audio'] = jumble_list
enc_ctrl3

,image_shown,stim1,animacy,condition,audio
14,crocodile,crocodile_2,animate,control,jumble1.mp3
15,cow,cow_2,animate,control,jumble2.mp3
16,chicken,chicken_1,animate,control,jumble3.mp3
17,cat,cat_1,animate,control,jumble4.mp3
18,bird,bird_1,animate,control,jumble5.mp3
19,bear,bear_1,animate,control,jumble6.mp3
20,bat,bat_1,animate,control,jumble7.mp3
14,door,door_1,inanimate,control,jumble8.mp3
15,cymbal_ride,cymbal_ride_2,inanimate,control,jumble9.mp3
16,car,car_3,inanimate,control,jumble10.mp3


## Put them all together
### 3 files for 3 conditions

In [138]:
version1enc = enc_cong1.append(enc_incong3).append(enc_ctrl2)
version1enc['phase'] = 'encoding'
version1enc['type'] = 'test'
version1enc['stimFormat'] = '.png'
version1enc['presTime'] = 400
version1enc['button1'] = 'animate'
version1enc['button2'] = 'inanimate'
version1enc = version1enc.reset_index()
version1enc.at[23,'audio'] = 'doorA.mp3'
version1enc.at[27,'audio'] = 'hammerA.mp3'
#version1enc = version1enc.sample(frac = 1)
version1enc

,index,image_shown,stim1,animacy,condition,audio,phase,type,stimFormat,presTime,button1,button2
0,0,wolf,wolf_1,animate,congruent,wolfA.mp3,encoding,test,.png,400,animate,inanimate
1,1,tiger,tiger_1,animate,congruent,tigerA.mp3,encoding,test,.png,400,animate,inanimate
2,2,snake,snake_1,animate,congruent,snakeA.mp3,encoding,test,.png,400,animate,inanimate
3,3,rat,rat_1,animate,congruent,ratA.mp3,encoding,test,.png,400,animate,inanimate
4,4,pig,pig_3,animate,congruent,pigA.mp3,encoding,test,.png,400,animate,inanimate
5,5,penguin,penguin_1,animate,congruent,penguinA.mp3,encoding,test,.png,400,animate,inanimate
6,6,leopard,leopard_1,animate,congruent,leopardA.mp3,encoding,test,.png,400,animate,inanimate
7,0,washer,washer_2,inanimate,congruent,washerA.mp3,encoding,test,.png,400,animate,inanimate
8,1,train,train_1,inanimate,congruent,trainA.mp3,encoding,test,.png,400,animate,inanimate
9,2,stapler,stapler_1,inanimate,congruent,staplerA.mp3,encoding,test,.png,400,animate,inanimate


In [139]:
version2enc = enc_cong2.append(enc_incong1).append(enc_ctrl3)
version2enc['phase'] = 'encoding'
version2enc['type'] = 'test'
version2enc['stimFormat'] = '.png'
version2enc['presTime'] = 400
version2enc['button1'] = 'animate'
version2enc['button2'] = 'inanimate'
version2enc = version2enc.reset_index()
version2enc.at[14,'audio'] = 'chickenA.mp3'
version2enc.at[16,'audio'] = 'crocodileA.mp3'
version2enc = version2enc.sample(frac = 1)
version2enc

,index,image_shown,stim1,animacy,condition,audio,phase,type,stimFormat,presTime,button1,button2
27,6,piano,piano_1,inanimate,incongruent,bellA.mp3,encoding,test,.png,400,animate,inanimate
12,12,drum,drum_2,inanimate,congruent,drumA.mp3,encoding,test,.png,400,animate,inanimate
11,11,guitar,guitar_3,inanimate,congruent,guitarA.mp3,encoding,test,.png,400,animate,inanimate
35,14,door,door_1,inanimate,control,jumble8.mp3,encoding,test,.png,400,animate,inanimate
10,10,hammer,hammer_3,inanimate,congruent,hammerA.mp3,encoding,test,.png,400,animate,inanimate
5,12,dog,dog_2,animate,congruent,dogA.mp3,encoding,test,.png,400,animate,inanimate
1,8,goose,goose_1,animate,congruent,gooseA.mp3,encoding,test,.png,400,animate,inanimate
6,13,deer,deer_4,animate,congruent,deerA.mp3,encoding,test,.png,400,animate,inanimate
40,19,bicycle,bicycle_1,inanimate,control,jumble13.mp3,encoding,test,.png,400,animate,inanimate
26,5,saw,saw_1,inanimate,incongruent,bicycleA.mp3,encoding,test,.png,400,animate,inanimate


In [140]:
version3enc = enc_cong3.append(enc_incong2).append(enc_ctrl1)
version3enc['phase'] = 'encoding'
version3enc['type'] = 'test'
version3enc['stimFormat'] = '.png'
version3enc['presTime'] = 400
version3enc['button1'] = 'animate'
version3enc['button2'] = 'inanimate'
version3enc = version3enc.reset_index()
version3enc.at[27,'audio'] = 'penguinA.mp3'
version3enc.at[20,'audio'] = 'snakeA.mp3'
version3enc.at[16,'audio'] = 'leopardA.mp3'
version3enc.at[19,'audio'] = 'pigA.mp3'
version3enc.at[22,'audio'] = 'staplerA.mp3'
version3enc.at[23,'audio'] = 'trainA.mp3'
version3enc = version3enc.sample(frac = 1)

### Recognition phase:
- copy each version separately
- remove audio
- phase = encoding
- new buttons (4)
- add in all the 'new' items
- shuffle again

In [141]:
new_cols = control_images[['object_image', 'stim1']]
new_cols = new_cols.rename(columns={'object_image': 'image_shown'})
new_cols['animacy'] = ['inanimate', 'animate', 'inanimate', 'inanimate', 'inanimate','animate','animate','animate','inanimate', 'inanimate', 'animate', 'inanimate', 'animate', 'animate', 'inanimate', 'animate', 'animate','inanimate']
new_cols['condition'] = 'new'
new_cols['phase'] = 'recognition'
new_cols['type'] = 'test'
new_cols['presTime'] = 400
new_cols['stimFormat'] = '.png'
new_cols['button1'] = 'same'
new_cols['button2'] = 'different'
new_cols['button3'] = 'nonsense'
new_cols['button4'] = 'new'
new_cols

,image_shown,stim1,animacy,condition,phase,type,presTime,stimFormat,button1,button2,button3,button4
0,wrench,wrench_1,inanimate,new,recognition,test,400,.png,same,different,nonsense,new
2,whale,whale_5,animate,new,recognition,test,400,.png,same,different,nonsense,new
3,watering_can,watering_can_1,inanimate,new,recognition,test,400,.png,same,different,nonsense,new
5,walkie,walkie_1,inanimate,new,recognition,test,400,.png,same,different,nonsense,new
8,tape,tape_1,inanimate,new,recognition,test,400,.png,same,different,nonsense,new
12,shark,shark_2,animate,new,recognition,test,400,.png,same,different,nonsense,new
15,salamander,salamander_3,animate,new,recognition,test,400,.png,same,different,nonsense,new
16,rhino,rhino_2,animate,new,recognition,test,400,.png,same,different,nonsense,new
17,record_player,record_player_1,inanimate,new,recognition,test,400,.png,same,different,nonsense,new
19,radio,radio_2,inanimate,new,recognition,test,400,.png,same,different,nonsense,new


### trial file 1

In [142]:
version1rec = version1enc.copy()
version1rec = version1rec.drop(columns=['audio'])
version1rec['phase'] = 'recognition'
version1rec['button1'] = 'same'
version1rec['button2'] = 'different'
version1rec['button3'] = 'nonsense'
version1rec['button4'] = 'new'

In [143]:
version1rec = version1rec.merge(new_cols, how='outer')
version1rec = version1rec.sample(frac = 1)

In [144]:
trial_file1 = version1enc.append(version1rec)
trial_file1['version'] = 1
trial_file1 = trial_file1[['version','type','phase','condition','animacy','image_shown','stim1','stimFormat','audio','presTime','button1','button2','button3','button4']]
#trial_file1.to_csv('/Users/sheaduarte/Desktop/trial_file1.csv', index=False)

### trial file 2

In [145]:
version2rec = version2enc.copy()
version2rec = version2rec.drop(columns=['audio'])
version2rec['phase'] = 'recognition'
version2rec['button1'] = 'same'
version2rec['button2'] = 'different'
version2rec['button3'] = 'nonsense'
version2rec['button4'] = 'new'

In [146]:
version2rec = version2rec.merge(new_cols, how='outer')
version2rec = version2rec.sample(frac = 1)

In [147]:
trial_file2 = version2enc.append(version2rec)
trial_file2['version'] = 2
trial_file2 = trial_file2[['version','type','phase','condition','animacy','image_shown','stim1','stimFormat','audio','presTime','button1','button2','button3','button4']]
#trial_file2.to_csv('/Users/sheaduarte/Desktop/trial_file2.csv', index=False)

### trial file 3

In [148]:
version3rec = version3enc.copy()
version3rec = version3rec.drop(columns=['audio'])
version3rec['phase'] = 'recognition'
version3rec['button1'] = 'same'
version3rec['button2'] = 'different'
version3rec['button3'] = 'nonsense'
version3rec['button4'] = 'new'

In [149]:
version3rec = version3rec.merge(new_cols, how='outer')
version3rec = version3rec.sample(frac = 1)

In [150]:
trial_file3 = version3enc.append(version3rec)
trial_file3['version'] = 3
trial_file3 = trial_file3[['version','type','phase','condition','animacy','image_shown','stim1','stimFormat','audio','presTime','button1','button2','button3','button4']]
#trial_file3.to_csv('/Users/sheaduarte/Desktop/trial_file3.csv', index=False)

In [151]:
len(trial_file2)

102

In [152]:
len(trial_file1)

102

In [153]:
trial_file1
#trial_file1.to_csv('/Users/sheaduarte/Desktop/trial_file1.csv', index=False)

,version,type,phase,condition,animacy,image_shown,stim1,stimFormat,audio,presTime,button1,button2,button3,button4
0,1,test,encoding,congruent,animate,wolf,wolf_1,.png,wolfA.mp3,400,animate,inanimate,NaN,NaN
1,1,test,encoding,congruent,animate,tiger,tiger_1,.png,tigerA.mp3,400,animate,inanimate,NaN,NaN
2,1,test,encoding,congruent,animate,snake,snake_1,.png,snakeA.mp3,400,animate,inanimate,NaN,NaN
3,1,test,encoding,congruent,animate,rat,rat_1,.png,ratA.mp3,400,animate,inanimate,NaN,NaN
4,1,test,encoding,congruent,animate,pig,pig_3,.png,pigA.mp3,400,animate,inanimate,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,1,test,recognition,new,animate,rhino,rhino_2,.png,NaN,400,same,different,nonsense,new
27,1,test,recognition,incongruent,inanimate,bell,bell_5,.png,NaN,400,same,different,nonsense,new
56,1,test,recognition,new,inanimate,coins,coins_1,.png,NaN,400,same,different,nonsense,new
48,1,test,recognition,new,animate,salamander,salamander_3,.png,NaN,400,same,different,nonsense,new


In [154]:
trial_file1.head(50)

,version,type,phase,condition,animacy,image_shown,stim1,stimFormat,audio,presTime,button1,button2,button3,button4
0,1,test,encoding,congruent,animate,wolf,wolf_1,.png,wolfA.mp3,400,animate,inanimate,NaN,NaN
1,1,test,encoding,congruent,animate,tiger,tiger_1,.png,tigerA.mp3,400,animate,inanimate,NaN,NaN
2,1,test,encoding,congruent,animate,snake,snake_1,.png,snakeA.mp3,400,animate,inanimate,NaN,NaN
3,1,test,encoding,congruent,animate,rat,rat_1,.png,ratA.mp3,400,animate,inanimate,NaN,NaN
4,1,test,encoding,congruent,animate,pig,pig_3,.png,pigA.mp3,400,animate,inanimate,NaN,NaN
5,1,test,encoding,congruent,animate,penguin,penguin_1,.png,penguinA.mp3,400,animate,inanimate,NaN,NaN
6,1,test,encoding,congruent,animate,leopard,leopard_1,.png,leopardA.mp3,400,animate,inanimate,NaN,NaN
7,1,test,encoding,congruent,inanimate,washer,washer_2,.png,washerA.mp3,400,animate,inanimate,NaN,NaN
8,1,test,encoding,congruent,inanimate,train,train_1,.png,trainA.mp3,400,animate,inanimate,NaN,NaN
9,1,test,encoding,congruent,inanimate,stapler,stapler_1,.png,staplerA.mp3,400,animate,inanimate,NaN,NaN


In [155]:
trial_file1.to_csv('/Users/sheaduarte/Desktop/trial_file1.csv', index=False)
trial_file2.to_csv('/Users/sheaduarte/Desktop/trial_file2.csv', index=False)
trial_file3.to_csv('/Users/sheaduarte/Desktop/trial_file3.csv', index=False)